# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,112.998334,-0.267183,1234,112.731151,113.265517,225.462303,226.531034
6,752.229946,-0.987676,1234,751.242270,753.217623,1502.484540,1506.435245
7,694.859348,0.831819,1234,695.691167,694.027530,1391.382334,1388.055060
8,224.002686,-0.929512,1234,223.073174,224.932198,446.146348,449.864397
9,258.825079,-0.612162,1234,258.212917,259.437241,516.425834,518.874482
10,904.175953,-0.700982,1234,903.474971,904.876935,1806.949942,1809.753869
11,164.964426,-0.960526,1234,164.003900,165.924951,328.007800,331.849903
12,886.030854,-0.990222,1234,885.040632,887.021076,1770.081264,1774.042152
13,457.871450,0.992371,1234,458.863821,456.879079,917.727643,913.758158
14,731.482197,0.958063,1234,732.440260,730.524134,1464.880519,1461.048267


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-112.731151,113.265517
6,-751.242270,753.217623
7,-695.691167,694.027530
8,-223.073174,224.932198
9,-258.212917,259.437241
10,-903.474971,904.876935
11,-164.003900,165.924951
12,-885.040632,887.021076
13,-458.863821,456.879079
14,-732.440260,730.524134


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-112.731151,113.265517
6,-751.242270,753.217623
7,-695.691167,694.027530
8,-223.073174,224.932198
9,-258.212917,259.437241
10,-903.474971,904.876935
11,-164.003900,165.924951
12,-885.040632,887.021076
13,-458.863821,456.879079
14,-732.440260,730.524134


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-112.73115125,  113.26551717],
       [-751.24226988,  753.2176227 ],
       [-695.69116701,  694.0275298 ],
       [-223.07317385,  224.93219836],
       [-258.21291718,  259.43724081],
       [-903.47497096,  904.87693453],
       [-164.00389994,  165.92495143],
       [-885.04063221,  887.02107579],
       [-458.86382141,  456.87907894],
       [-732.44025954,  730.52413355]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,223.462303,-226.531034
6,1500.484540,-1506.435245
7,1389.382334,-1388.055060
8,444.146348,-449.864397
9,514.425834,-518.874482
10,1804.949942,-1809.753869
11,326.007800,-331.849903
12,1768.081264,-1774.042152
13,915.727643,-913.758158
14,1462.880519,-1461.048267


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
